# 실행환경:
Colab Pro Plus - A100 GPU

# 경로 지정하기, 데이터셋 불러오기, 라이브러리 설치하기

본 베이스라인 코드는 **모든 참가자들이 최소한의 제출을 할 수 있도록** 간단하게 만든 코드입니다. 이 코드를 토대로 자유롭게 수정하거나 다른 아이디어를 추가하여도 괜찮습니다. 당연히 베이스라인 코드를 참고하지 않고 처음부터 코드를 작성하여도 무방합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# 구글 드라이브를 코랩과 연결하는 코드입니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 현재 경로를 지정하는 코드입니다.
%cd '/content/drive/MyDrive/hi/'

/content/drive/MyDrive/hi


In [5]:
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [6]:
# 베이스라인 코드를 실행하기 위하여 필요한 라이브러리를 설치하는 코드입니다.
!pip install sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib jsonlines rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.3 MB/s eta 0:00:00


jsonl 형식의 데이터셋을 불러오는 코드입니다.

    {
      "id": 1,
      "type": "Type 2",
      "size": 6,
      "set": "또#이상#준다#핀잔#사#티셔츠",
      "sentence": "또 이상한 티셔츠를 사서 핀잔을 준다."
    },
    {
      "id": 2,
      "type": "Type 1",
      "size": 6,
      "set": "탄다#해변#썰물 때#말#사람#세",
      "sentence": "썰물 때에 세명의 사람들이 해변에서 말을 탄다."
    },
    ...

데이터의 형태는 위와 같습니다. 가장 간단하게 `"set"`을 입력으로, `"sentence"`를 출력으로 사용할 것입니다. 이후 훈련 집합을 `train.jsonl`에 저장하였습니다.
또한 위에서 `"sentence"`가 빠진 형태인 평가 집합을 `eval.jsonl`에 저장하였습니다.

다른 변수들도 학습에 도움이 될 것이라고 판단하신다면 자유롭게 수정하셔도 좋습니다. 여러분의 아이디어를 보여 주세요!

In [ ]:
import jsonlines
import json

# 데이터셋을 위의 설명대로 변환하는 코드입니다.
input_file_path = 'train.json'
train_file_path = 'train.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(train_file_path, mode='w') as writer:
    for item in data['dataset']:
        writer.write({"input": item["set"], "output": item["sentence"]})

input_file_path = 'test.json'
test_file_path = 'test.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(input_file_path, mode='w') as writer:
    for item in data['dataset']:
        writer.write({"input": item["set"]})

(train_file_path, test_file_path)

In [12]:
# Custom

input_file_path = 'split_train.json'
train_file_path = 'train_split.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(train_file_path, mode='w') as writer:
    for item in data:
        writer.write({"input": item["set"], "output": item["sentence"]})

In [ ]:
# Custom

input_file_path = 'split_valid.json'
train_file_path = 'eval.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(train_file_path, mode='w') as writer:
    for item in data:
        writer.write({"input": item["set"], "output": item["sentence"]})

In [4]:
# 데이터셋을 불러오는 코드입니다.
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='train_split.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='eval.jsonl', split='train')
test_dataset = load_dataset('json', data_files='test.jsonl', split='train')

In [6]:
train_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 27000
})

# 베이스 모델 로드

Hugging Face의 Transformers 라이브러리를 사용하여 사전 훈련된 언어 모델을 로드합니다. 베이스라인에는 `
EleutherAI/polyglot-ko-1.3b`라는 언어 모델을 사용하였습니다.

In [39]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
# del model

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
tokenizer.sep_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [22]:
tokenizer.sep_token = tokenizer.eos_token

In [17]:
train_dataset[0]['input'].split('#')

['한다', '준비', '가위', '종이', '자를']

In [18]:
# Training code using bart model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

def preprocess_function(examples):
    inputs = [tokenizer.eos_token.join(doc.split('#')) for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=64, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=256, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(
    preprocess_function, batched=True, num_proc=4, load_from_cache_file=True
)
tokenized_eval_dataset = eval_dataset.map(
    preprocess_function, batched=True, num_proc=4, load_from_cache_file=True)

In [19]:
print(tokenized_train_dataset[0])
print(tokenized_eval_dataset[0])

{'input': '한다#준비#가위#종이#자를', 'output': '가위가 종이를 자를 준비를 한다.', 'input_ids': [16501, 1, 14846, 1, 14039, 11973, 1, 22133, 1, 29871, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [14039, 15795, 14317, 15188, 29871, 20227, 19553, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [20]:
tokenizer.decode(tokenized_train_dataset[0]['input_ids'])

'한다</s> 준비</s> 가위</s> 종이</s> 자를<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Training

In [40]:
args = Seq2SeqTrainingArguments(
    "kobart",
    num_train_epochs=10,
    learning_rate=5e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    warmup_steps=400,
    save_total_limit=3,
    remove_unused_columns=True,
    logging_strategy="steps",
    logging_steps=300,
    evaluation_strategy = "steps",
    eval_steps=300,
    save_strategy="steps",
    save_steps=300,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss
300,0.052100,0.042817
600,0.046000,0.042081
900,0.035300,0.043619


KeyboardInterrupt: ignored

# Evaluation (Unused)

In [ ]:
# 생성된 텍스트를 평가하는 코드입니다.
from rouge import Rouge
from bert_score import score

predictions_file = 'predictions_baseline_trained.jsonl'
eval_file = 'train.jsonl'

predictions = []
references = []
with jsonlines.open(predictions_file) as pred_reader, jsonlines.open(eval_file) as eval_reader:
    for i, (pred, eval) in enumerate(zip(pred_reader, eval_reader)):
        if i >= 100:
            break
        predictions.append(''.join(pred["output"].split(pred["input"])[1:]))
        references.append(eval["output"])

rouge = Rouge()
rouge_scores = rouge.get_scores(predictions, references, avg=True)

P, R, F1 = score(predictions, references, lang="ko", model_type="bert-base-multilingual-cased")
bertscore = F1.mean()

print("ROUGE-1:", round(rouge_scores['rouge-1']['f'], 4))
print("ROUGE-2:", round(rouge_scores['rouge-2']['f'], 4))
print("ROUGE-L:", round(rouge_scores['rouge-l']['f'], 4))
print("BERTScore:", round(bertscore.item(), 4))

print()
print("Final Score:", round(0.1 * rouge_scores['rouge-1']['f'] + 0.1 * rouge_scores['rouge-2']['f'] +
                            0.2 * rouge_scores['rouge-l']['f'] + 0.6 * bertscore.item(), 4))

# Inference

In [55]:
ft_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/hi/kobart/checkpoint-900")
ft_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/hi/kobart/checkpoint-900")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
from tqdm import tqdm
import torch

def process_func(text):
  return ft_tokenizer.eos_token.join(text.split('#'))

outputs = []
for data in tqdm(test_dataset):
  inputs = ft_tokenizer(process_func(data['input']), return_tensors="pt")

  inputs = {k: v.to("cuda") for k, v in inputs.items()}
  generated_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=30, no_repeat_ngram_size=2)
  generated_texts = ft_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  outputs.append(generated_texts)
  # print(f"gt: {data['output']}")
  print(f"output: {generated_texts}")

# Submission

In [69]:
import pandas as pd
results = []

idx = 30001

for data in outputs:
  results.append(dict(
      id=idx,
      sentence=data[0].split('.')[0] + '.'
  ))
  idx += 1

df = pd.DataFrame(results, columns=['id', 'sentence'])
df

,id,sentence
0,30001,나무 트레일러 옆에 배들이 정박해 있었다.
1,30002,연락이 딱 마주쳐서 그냥 상수라고 했다.
2,30003,술은 약해서 술을 안 먹는다고 하였다.
3,30004,어른과 건물 밖에서 남자와 용 연을 함께 든 남자가 나간다 하였다.
4,30005,두마리의 새가 섬의 주차장 근처의 풀에서 풀을 먹고 있다.
...,...,...
1495,31496,두개의 파란 색 배경 위에 전화기가 거의 놓여 있다.
1496,31497,사진을 보니면서 탈모가 걱정된다고 걱정하고 있다.
1497,31498,술과 여자 몇명이 파티에서 술을 즐기고 있다.
1498,31499,꽃이 피기 시작하는 큰 잎 식물이 야외에서 꽃을 피는 날로 시작되었다고 한다.


In [70]:
# 제출 파일을 로컬이나 구글 드라이브에 저장하는 코드입니다.
df.to_csv('last_dance.csv', index=False, encoding='utf-8')